In [1]:
from aidream_registration import constants
import aidream_registration.utils.cohort_utils as cu

from aidream_registration.labels_registration.native_label_registration import NativeLabelRegistration
from aidream_registration.labels_registration.processed_label_registration import ProcessedLabelRegistration


In [3]:
# the list of perfusion patients :
list_patients = cu.get_perfusion_patients()
print(fr"Number of patients: {len(list_patients)}")


Number of patients: 186


In [4]:
# the referential table :
df_ref = cu.get_referential_table(list_patients)
df_ref.head(5)


,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_reference,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
0,AIDREAM_1,MMI-PROB_140,201211519BP,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes
1,AIDREAM_10,MMI-PROB_050,201002036RR,yes,1,2.0,AC_repATLAS native,RS,yes,yes,AC_repATLAS,RS,yes,yes
2,AIDREAM_100,MMI-PROB_139,201210738EE,yes,1,0.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,no
3,AIDREAM_102,MMI-PROB_085,201101726ZB,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes
4,AIDREAM_103,MMI-PROB_166,201311898RR,yes,1,2.0,AC_repATLAS,RS,yes,yes,AC_repATLAS,RS,yes,yes


In [5]:
# The Native label registerer :
native_label_registerer = NativeLabelRegistration()

# The Processed label registerer :
processed_label_registerer = ProcessedLabelRegistration()


# 1. First, register pre_RT labels : #

In [6]:
dir_workspace = constants.DIR_DEFAULT_HARD_DRIVE / "AIDREAM DATA" / "LABELS DATA" / "REGISTERED LABELS ON PRE_RT T1"
dir_workspace.mkdir(parents=True, exist_ok=True)

list_pre_rt_labels = ["L1", "L2", "L3", "L4", "L5"]


In [7]:
df_ref.groupby("pre_RT_reference").count()

,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_reference,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
pre_RT_reference,,,,,,,,,,,,,
AC_repATLAS,119,119,119,119,119,119,119,119,119,117,119,119,119
AC_repATLAS native,1,1,1,1,1,1,1,1,1,1,1,1,1
GE_repNATIV,38,38,38,38,38,38,38,38,38,36,38,38,38
GE_repNativAXIALISEDnew,9,9,9,9,9,9,9,9,9,7,9,9,9
GE_repNativnew,12,12,12,12,12,12,12,12,12,12,12,12,12
"JBD : Quel repère ? Pas de crâne, jcomprends pas",1,1,1,1,1,1,1,1,1,1,1,1,1
Repère natif,1,1,1,1,1,1,1,1,1,1,1,1,1
skullstripaxialized RS recalé sur GE,5,5,5,5,5,5,5,5,5,5,5,5,5


### 1.1. AC_repATLAS patients :

In [8]:
# First, the patients with AC_repATLAS reference :
ref = "AC_repATLAS"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="pre_RT",
                                                                     reference=ref,
                                                                     list_labels=list_pre_rt_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)


Number of patients with AC_repATLAS reference: 119
Processing 1/119: AIDREAM_1 .........
Loading AIDREAM_1 ATLAS pre_RT T1...
Loading AIDREAM_1 pre_RT source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Patient AIDREAM_1 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_1 pre_RT ...
Patient AIDREAM_1 doesn't have pre_RT L3, saving empty label !
Applying affine transformation to L4 for AIDREAM_1 pre_RT ...
Applying affine transformation to L5 for AIDREAM_1 pre_RT ...
Loading AIDREAM_1 ATLAS pre_RT T1CE...
Loading AIDREAM_1 pre_RT Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Patient AIDREAM_1 doesn't have pre_RT L1, saving empty label !
Applying SyN transformation to L2 for AIDREAM_1 pre_RT ...
Patient AIDREAM_1 doesn't have pre_RT L3, saving empty label

### 1.2. AC_repATLAS native patients :

In [9]:
# For the AC_repATLAS native patients, we apply native label registration :
ref = "AC_repATLAS native"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    native_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)


Number of patients with AC_repATLAS native reference: 1
Processing 1/1: AIDREAM_10 .........
Loading AIDREAM_10 ATLAS pre_RT T1...
Patient AIDREAM_10 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_10 pre_RT ...
Patient AIDREAM_10 doesn't have pre_RT L3, saving empty label !
Applying affine transformation to L4 for AIDREAM_10 pre_RT ...
Applying affine transformation to L5 for AIDREAM_10 pre_RT ...


### 1.3. GE_repNATIV native patients :

In [10]:
# For the GE_repNATIV patients, we apply native label registration :
ref = "GE_repNATIV"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    native_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)


Number of patients with GE_repNATIV reference: 38
Processing 1/38: AIDREAM_200 .........
Loading AIDREAM_200 ATLAS pre_RT T1...
Patient AIDREAM_200 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_200 pre_RT ...
Patient AIDREAM_200 doesn't have pre_RT L3, saving empty label !
Applying affine transformation to L4 for AIDREAM_200 pre_RT ...
Applying affine transformation to L5 for AIDREAM_200 pre_RT ...
Processing 2/38: AIDREAM_201 .........
Loading AIDREAM_201 ATLAS pre_RT T1...
Patient AIDREAM_201 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_201 pre_RT ...
Patient AIDREAM_201 doesn't have pre_RT L3, saving empty label !
Applying affine transformation to L4 for AIDREAM_201 pre_RT ...
Applying affine transformation to L5 for AIDREAM_201 pre_RT ...
Processing 3/38: AIDREAM_203 .........
Loading AIDREAM_203 ATLAS pre_RT T1...
Applying affine transformation to L1 for AIDREAM_203 pre_RT ...
Applying

### 1.4 GE_repNativAXIALISEDnew patients :


In [11]:
ref = "GE_repNativAXIALISEDnew"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="pre_RT",
                                                                     reference=ref,
                                                                     list_labels=list_pre_rt_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)



Number of patients with GE_repNativAXIALISEDnew reference: 9
Processing 1/9: AIDREAM_120 .........
Loading AIDREAM_120 ATLAS pre_RT T1...
Loading AIDREAM_120 pre_RT source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Patient AIDREAM_120 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_120 pre_RT ...
Applying affine transformation to L3 for AIDREAM_120 pre_RT ...
Patient AIDREAM_120 doesn't have pre_RT L4, saving empty label !
Applying affine transformation to L5 for AIDREAM_120 pre_RT ...
Loading AIDREAM_120 ATLAS pre_RT T1CE...
Loading AIDREAM_120 pre_RT Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Patient AIDREAM_120 doesn't have pre_RT L1, saving empty label !
Applying SyN transformation to L2 for AIDREAM_120 pre_RT ...
Applying SyN transformation 

### 1.5. GE_repNativnew patients :

In [12]:
ref = "GE_repNativnew"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="pre_RT",
                                                                     reference=ref,
                                                                     list_labels=list_pre_rt_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)


Number of patients with GE_repNativnew reference: 12
Processing 1/12: AIDREAM_202 .........
Loading AIDREAM_202 ATLAS pre_RT T1...
Loading AIDREAM_202 pre_RT source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Patient AIDREAM_202 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_202 pre_RT ...
Applying affine transformation to L3 for AIDREAM_202 pre_RT ...
Applying affine transformation to L4 for AIDREAM_202 pre_RT ...
Applying affine transformation to L5 for AIDREAM_202 pre_RT ...
Loading AIDREAM_202 ATLAS pre_RT T1CE...
Loading AIDREAM_202 pre_RT Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Patient AIDREAM_202 doesn't have pre_RT L1, saving empty label !
Applying SyN transformation to L2 for AIDREAM_202 pre_RT ...
Applying SyN transformation to L3 fo

### 1.6. JBD : Quel repère ? Pas de crâne, jcomprends pas  patients :

In [20]:
ref = "JBD : Quel repère ? Pas de crâne, jcomprends pas"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

ref = "JBD"

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="pre_RT",
                                                                     reference=ref,
                                                                     list_labels=list_pre_rt_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)


Number of patients with JBD reference: 1
Processing 1/1: AIDREAM_357 .........
Loading AIDREAM_357 ATLAS pre_RT T1...
Loading AIDREAM_357 pre_RT source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Patient AIDREAM_357 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_357 pre_RT ...
Applying affine transformation to L3 for AIDREAM_357 pre_RT ...
Applying affine transformation to L4 for AIDREAM_357 pre_RT ...
Applying affine transformation to L5 for AIDREAM_357 pre_RT ...
Loading AIDREAM_357 ATLAS pre_RT T1CE...
Loading AIDREAM_357 pre_RT Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Patient AIDREAM_357 doesn't have pre_RT L1, saving empty label !
Applying SyN transformation to L2 for AIDREAM_357 pre_RT ...
Applying SyN transformation to L3 for AIDREAM_357

### 1.7 repère natif patients :

In [25]:
ref = "Repère natif"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

ref = "Repere natif"
print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    native_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)


Number of patients with Repere natif reference: 1
Processing 1/1: AIDREAM_116 .........
Loading AIDREAM_116 ATLAS pre_RT T1...
Patient AIDREAM_116 doesn't have pre_RT L1, saving empty label !
Applying affine transformation to L2 for AIDREAM_116 pre_RT ...
Patient AIDREAM_116 doesn't have pre_RT L3, saving empty label !
Applying affine transformation to L4 for AIDREAM_116 pre_RT ...
Applying affine transformation to L5 for AIDREAM_116 pre_RT ...


### 1.8. skullstripaxialized RS recalé sur GE patients :

In [27]:
ref = "skullstripaxialized RS recalé sur GE"
list_patients = df_ref.loc[df_ref["pre_RT_reference"] == ref, "AIDREAM_ID"].tolist()

ref = "skullstripaxialized RS recale sur GE"

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="pre_RT",
                                                                     reference=ref,
                                                                     list_labels=list_pre_rt_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="pre_RT",
                                                                  reference=ref,
                                                                  list_labels=list_pre_rt_labels)


Number of patients with skullstripaxialized RS recale sur GE reference: 5
Processing 1/5: AIDREAM_226 .........
Loading AIDREAM_226 ATLAS pre_RT T1...
Loading AIDREAM_226 pre_RT source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Applying affine transformation to L1 for AIDREAM_226 pre_RT ...
Applying affine transformation to L2 for AIDREAM_226 pre_RT ...
Applying affine transformation to L3 for AIDREAM_226 pre_RT ...
Patient AIDREAM_226 doesn't have pre_RT L4, saving empty label !
Patient AIDREAM_226 doesn't have pre_RT L5, saving empty label !
Loading AIDREAM_226 ATLAS pre_RT T1CE...
Loading AIDREAM_226 pre_RT Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Applying SyN transformation to L1 for AIDREAM_226 pre_RT ...
Applying SyN transformation to L2 for AIDREAM_226 pre_RT ...
Applying SyN transf

# 2. Second, register Rechute labels :

In [28]:
dir_workspace = constants.DIR_DEFAULT_HARD_DRIVE / "AIDREAM DATA" / "LABELS DATA" / "REGISTERED LABELS ON PRE_RT T1"
dir_workspace.mkdir(parents=True, exist_ok=True)

list_rechute_labels = ["L1R", "L2R", "L3R", "L4R", "L5R"]


In [29]:
df_ref.groupby("Rechute_reference").count()


,AIDREAM_ID,cohort_ID,local_ID,perfusion,complete_validation,surgery_type,pre_RT_reference,pre_RT_TPS,valid_pre_RT_segmentation_model,valid_pre_RT_perfusion_model,Rechute_TPS,valid_Rechute_segmentation_model,valid_Rechute_perfusion_model
Rechute_reference,,,,,,,,,,,,,
AC_repATLAS,101,101,101,101,101,101,101,101,101,101,101,101,101
GE_repNativAXIALISEDnew,68,68,68,68,68,68,68,68,68,68,68,68,68
Repère natif,4,4,4,4,4,4,4,4,4,4,4,4,4
skullstripaxialized RS recalé sur GE,7,7,7,7,7,7,7,7,7,7,7,7,7


### 2.1. AC_repATLAS patients :

In [31]:
# First, the patients with AC_repATLAS reference :
ref = "AC_repATLAS"
list_patients = df_ref.loc[df_ref["Rechute_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="Rechute",
                                                                     reference=ref,
                                                                     list_labels=list_rechute_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="Rechute",
                                                                  reference=ref,
                                                                  list_labels=list_rechute_labels)


Number of patients with AC_repATLAS reference: 101
Processing 1/101: AIDREAM_1 .........
Loading AIDREAM_1 ATLAS pre_RT T1...
Loading AIDREAM_1 Rechute source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Applying affine transformation to L1R for AIDREAM_1 Rechute ...
Applying affine transformation to L2R for AIDREAM_1 Rechute ...
Applying affine transformation to L3R for AIDREAM_1 Rechute ...
Applying affine transformation to L4R for AIDREAM_1 Rechute ...
Applying affine transformation to L5R for AIDREAM_1 Rechute ...
Loading AIDREAM_1 ATLAS pre_RT T1CE...
Loading AIDREAM_1 Rechute Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Applying SyN transformation to L1R for AIDREAM_1 Rechute ...
Applying SyN transformation to L2R for AIDREAM_1 Rechute ...
Applying SyN transformation to L3R for AIDREAM_1 R

### 2.2. GE_repNativAXIALISEDnew patients :

In [36]:
# First, the patients with AC_repATLAS reference :
ref = "GE_repNativAXIALISEDnew"
list_patients = df_ref.loc[df_ref["Rechute_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="Rechute",
                                                                     reference=ref,
                                                                     list_labels=list_rechute_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="Rechute",
                                                                  reference=ref,
                                                                  list_labels=list_rechute_labels)

Number of patients with GE_repNativAXIALISEDnew reference: 68
Processing 1/68: AIDREAM_104 .........
Loading AIDREAM_104 ATLAS pre_RT T1...
Loading AIDREAM_104 Rechute source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Applying affine transformation to L1R for AIDREAM_104 Rechute ...
Applying affine transformation to L2R for AIDREAM_104 Rechute ...
Applying affine transformation to L3R for AIDREAM_104 Rechute ...
Patient AIDREAM_104 doesn't have Rechute L4R, saving empty label !
Patient AIDREAM_104 doesn't have Rechute L5R, saving empty label !
Loading AIDREAM_104 ATLAS pre_RT T1CE...
Loading AIDREAM_104 Rechute Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Applying SyN transformation to L1R for AIDREAM_104 Rechute ...
Applying SyN transformation to L2R for AIDREAM_104 Rechute ...
Applying SyN t

### 2.3. Repère natif patients :

In [37]:
ref = "Repère natif"
list_patients = df_ref.loc[df_ref["Rechute_reference"] == ref, "AIDREAM_ID"].tolist()

ref = "Repere natif"
print(fr"Number of patients with {ref} reference: {len(list_patients)}")

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    native_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                  stage="Rechute",
                                                                  reference=ref,
                                                                  list_labels=list_rechute_labels)


Number of patients with Repere natif reference: 4
Processing 1/4: AIDREAM_116 .........
Loading AIDREAM_116 ATLAS pre_RT T1...
Applying affine transformation to L1R for AIDREAM_116 Rechute ...
Applying affine transformation to L2R for AIDREAM_116 Rechute ...
Applying affine transformation to L3R for AIDREAM_116 Rechute ...
Applying affine transformation to L4R for AIDREAM_116 Rechute ...
Applying affine transformation to L5R for AIDREAM_116 Rechute ...
Processing 2/4: AIDREAM_248 .........
Loading AIDREAM_248 ATLAS pre_RT T1...
Applying affine transformation to L1R for AIDREAM_248 Rechute ...
Applying affine transformation to L2R for AIDREAM_248 Rechute ...
Applying affine transformation to L3R for AIDREAM_248 Rechute ...
Applying affine transformation to L4R for AIDREAM_248 Rechute ...
Applying affine transformation to L5R for AIDREAM_248 Rechute ...
Processing 3/4: AIDREAM_356 .........
Loading AIDREAM_356 ATLAS pre_RT T1...
Applying affine transformation to L1R for AIDREAM_356 Rechu

## 2.4. skullstripaxialized RS recalé sur GE patients :

In [39]:
ref = "skullstripaxialized RS recalé sur GE"

list_patients = df_ref.loc[df_ref["Rechute_reference"] == ref, "AIDREAM_ID"].tolist()

print(fr"Number of patients with {ref} reference: {len(list_patients)}")

ref = "skullstripaxialized RS recale sur GE"

for i, patient in enumerate(list_patients):

    print(fr"Processing {i + 1}/{len(list_patients)}: {patient} .........")

    processed_label_registerer.apply_affine_transformation_to_labels(patient=patient,
                                                                     stage="Rechute",
                                                                     reference=ref,
                                                                     list_labels=list_rechute_labels)

    processed_label_registerer.apply_syn_transformation_to_labels(patient=patient,
                                                                  stage="Rechute",
                                                                  reference=ref,
                                                                  list_labels=list_rechute_labels)



Number of patients with skullstripaxialized RS recalé sur GE reference: 7
Processing 1/7: AIDREAM_201 .........
Loading AIDREAM_201 ATLAS pre_RT T1...
Loading AIDREAM_201 Rechute source imaging...
Registering the source imaging on the ATLAS pre_RT T1...
Warping the source imaging using the affine transformation...
Patient AIDREAM_201 doesn't have Rechute L1R, saving empty label !
Applying affine transformation to L2R for AIDREAM_201 Rechute ...
Patient AIDREAM_201 doesn't have Rechute L3R, saving empty label !
Patient AIDREAM_201 doesn't have Rechute L4R, saving empty label !
Patient AIDREAM_201 doesn't have Rechute L5R, saving empty label !
Loading AIDREAM_201 ATLAS pre_RT T1CE...
Loading AIDREAM_201 Rechute Affine registered source imaging...
Registering the source imaging on the ATLAS pre_RT T1CE...
Warping the source imaging using the SyN transformation...
Patient AIDREAM_201 doesn't have Rechute L1R, saving empty label !
Applying SyN transformation to L2R for AIDREAM_201 Rechute .